In [38]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def image_mask():
    print("画像54枚にマスク画像を合成するよ！")
    for i in range(1, 55):
        #画像の読み込み
        #背景は、PRADAの服、前景はマスク画像(透過で読み込み)
        src  = cv2.imread("/Users/MorikawaLab/Desktop/miyakoda_data/masked_image/mask_zikken/PRADA_mask_haba55_up35_down50.png", -1)       #マスク画像. ここを変える必要あり.
        dst  = cv2.imread("/Users/MorikawaLab/Desktop/miyakoda_data/PRADA/PRADA ({0:d}).jpg".format(i), -1) #ファッション画像
        dst_ = dst.astype(np.float64)

        #前景の画像の大きさを調べる。
        width  = src.shape[0]    #幅
        height  = src.shape[1]   #高さ

        #マスク画像の作成
        mask = src[:,:,3]                                #これで４番目のアルファチャンネルの行列だけが抽出。配列のスライスの参考→https://qiita.com/okkn/items/54e81346d8f35733ab5e
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)   #maskを3色分にした。0 ->(B0, G0, R0)になる。
        mask = mask/250                                #正規化

        #画像の合成
        src = src[:,:, :3]
        dst_ *= 1 - mask                               # 透過率に応じて元の画像を暗くする。
        # dst_[:,:, :3] = (1 - mask) * dst_[:,:, :3]
        dst_ += src * mask
        cv2.imwrite("/Users/MorikawaLab/Desktop/miyakoda_data/PRADA_masked_data/PRADA_masked_image/PRADA_masked ({0:d}).png".format(i), dst_)


def color_to_table():
    print("これから画像の画素の操作をします")

    for i in range(1, 55):
        dst_ = cv2.imread("/Users/MorikawaLab/Desktop/miyakoda_data/PRADA_masked_data/PRADA_masked_image/PRADA_masked ({0:d}).png".format(i), -1)
    #     切り抜いた画像の色相分布について
    #     その後、色相・彩度・明度の分布を行列化。  

        height, width = dst_.shape[ :2]

        black = 34371 #計算で求めた. 詳しくはノート見て.

#         for j in range(0, height):
#             for k in range(0, width):

#                 "GIMPの提示する黒は(0, 0 ,0)なのでマスク部分をカウント"
#                 if dst_.item(j, k, 0)==0 and dst_.item(j, k, 1)==0 and dst_.item(j, k, 2)==0:
#                     black = black + 1

    #     print(i, "番目の黒色の数は",black )

        img_after =  cv2.cvtColor(dst_, cv2.COLOR_BGR2HSV)
        hist_m0= cv2.calcHist([img_after], [0], None, [180], [0, 180]) 
#         hist_m0[0] -= black
        hist_m1 = cv2.calcHist([img_after], [1], None, [256], [0, 256]) 
        hist_m1[0] -= black
        hist_m2 = cv2.calcHist([img_after], [2], None, [256], [0,256])
        hist_m2[0] -=  black

        np.savetxt('/Users/MorikawaLab/Desktop/miyakoda_data/PRADA_masked_data/PRADA_masked_hue/PRADA_masked_hue({0:d}).csv'.format(i), hist_m0, delimiter=',')
        np.savetxt('/Users/MorikawaLab/Desktop/miyakoda_data/PRADA_masked_data/PRADA_masked_sat/PRADA_masked_sat({0:d}).csv'.format(i), hist_m1, delimiter=',')
        np.savetxt('/Users/MorikawaLab/Desktop/miyakoda_data/PRADA_masked_data/PRADA_masked_val/PRADA_masked_val({0:d}).csv'.format(i), hist_m2, delimiter=',')
        if (i==54):
            print("画像の総ピクセル数は"+str(width * height)+"pxだ")
            print("マスク画像の色相・彩度・明度の行列化が終わりました")




def analyze_color():
    print("行列データを読み込み、区分分けして、割合の式にあてはめ、一致率を算出")
    correct_hue_total = 0
    correct_sat_total = 0
    correct_val_total = 0

    not_black = (width * height) - black

    for i in range(1, 4):
        #色相の色を読みこむ
        data1 = np.loadtxt('/Users/facul/Pictures/20180814_PRADA_masked/PRADA_hue({0:d}).csv'.format(i), delimiter=",") 

        hue_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

        hue10 = (data1[171: 180].sum() + data1[2: 10].sum()) / not_black
        hue_list[0] = round(hue10, 2)                                        #round(数値, 2)で小数点2ケタに丸める的な
        hue11 = data1[10:27].sum()  / not_black
        hue_list[1] = round(hue11, 2)
        hue12 = data1[27: 45].sum() /  not_black
        hue_list[2] = round(hue12, 2)
        hue13 = data1[45: 63].sum() /  not_black
        hue_list[3] = round(hue13, 2)
        hue14 = data1[63: 81].sum() /  not_black
        hue_list[4] = round(hue14, 2)
        hue15 = data1[81: 99].sum() /  not_black
        hue_list[5] = round(hue15, 2)
        hue16 = data1[99: 117].sum() /  not_black
        hue_list[6] = round(hue16, 2)
        hue17 = data1[117: 135].sum() /  not_black
        hue_list[7] = round(hue17, 2)
        hue18 = data1[135: 153].sum() /  not_black
        hue_list[8] = round(hue18, 2)
        hue19 = data1[153: 171].sum() /  not_black
        hue_list[9] = round(hue19, 2)

        print(hue_list)

        #彩度の色を読み込む
        data1 = np.loadtxt('/Users/facul/Pictures/20180814_PRADA_masked/PRADA_sat({0:d}).csv'.format(i), delimiter=",")    

        sat_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

        sat10 = data1[2: 18].sum() / not_black
        sat_list[0] = round(hue10, 2)                                        #round(数値, 2)で小数点2ケタに丸める的な
        sat11 = data1[18: 36].sum() / not_black
        sat_list[1] = round(hue11, 2)
        sat12 = data1[36: 54].sum() /  not_black
        sat_list[0] = round(hue10, 2)
        sat13 = data1[54: 72].sum() /  not_black
        sat_list[0] = round(hue10, 2)
        sat14 = data1[72: 90].sum() /  not_black
        hue_list[4] = round(hue14, 2)
        sat15 = data1[90: 108].sum() /  not_black
        sat_list[5] = round(hue15, 2)
        sat16 = data1[108: 126].sum() /  not_black
        sat_list[6] = round(hue16, 2)
        sat17 = data1[126: 144].sum() /  not_black
        sat_list[7] = round(hue17, 2)
        sat18 = data1[144: 162].sum() /  not_black
        sat_list[8] = round(hue18, 2)
        sat19 = data1[162: 180].sum() /  not_black
        sat_list[9] = round(hue19, 2)

        print(sat_list)

        #明度を読み込む
        data1 = np.loadtxt('/Users/facul/Pictures/20180814_PRADA_masked/PRADA_sat({0:d}).csv'.format(i), delimiter=",")

        val_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

        val10 = data1[2: 18].sum() / not_black
        val_list[0] = round(hue10, 2)                                        #round(数値, 2)で小数点2ケタに丸める的な
        val11 = data1[18: 36].sum() / not_black
        val_list[1] = round(hue11, 2)
        val12 = data1[36: 54].sum() /  not_black
        val_list[0] = round(hue10, 2)
        val13 = data1[54: 72].sum() /  not_black
        val_list[0] = round(hue10, 2)
        val14 = data1[72: 90].sum() /  not_black
        val_list[4] = round(hue14, 2)
        val15 = data1[90: 108].sum() /  not_black
        val_list[5] = round(hue15, 2)
        val16 = data1[108: 126].sum() /  not_black
        val_list[6] = round(hue16, 2)
        val17 = data1[126: 144].sum() /  not_black
        val_list[7] = round(hue17, 2)
        val18 = data1[144: 162].sum() /  not_black
        val_list[8] = round(hue18, 2)
        val19 = data1[162: 180].sum() /  not_black
        val_list[9] = round(hue19, 2)

        print(sat_list)

画像処理を複数の関数  
* image_mask()
* color_to_table()
* analyze_color()  
に分割した  
こうすることで, 毎回起動しなくてもよくなる.    
以下で実際に動かしてみる.

In [5]:
image_mask()   #20181003 動作確認済み

画像54枚にマスク画像を合成するよ！


In [37]:
color_to_table() 

これから画像の画素の操作をします
画像の総ピクセル数は43776pxだ
マスク画像の色相・彩度・明度の行列化が終わりました


In [39]:
analyze_color()

行列データを読み込み、区分分けして、割合の式にあてはめ、一致率を算出


NameError: name 'width' is not defined